# 유사 이미지 검출

In [49]:
from PIL import Image
import numpy as np
import os, re

In [5]:
img = Image.open('/Users/lesson6_mac/AtomData/python/img/ele.JPG')

In [6]:
img.size

(600, 337)

In [7]:
img.format_description

'JPEG (ISO 10918)'

In [9]:
img.show()

In [13]:
img2 = img.convert('L').resize((300,300), Image.ANTIALIAS)
img2.save('./img/ele_grey.JPG')

In [14]:
img2.show()

In [19]:
img3 = img.crop()
img3.show()

In [21]:
img4= img.convert('L')
pixels = np.array(img4) 
print(pixels.shape)
img4= img.convert('RGB')
pixels = np.array(img4) 
print(pixels.shape)
img4= img.convert('RGBA')
pixels = np.array(img4) 
print(pixels.shape)
img4= img.convert('CMYK')
pixels = np.array(img4) 
print(pixels.shape)

(337, 600)
(337, 600, 3)
(337, 600, 4)
(337, 600, 4)


- 리스트 보단 튜플이 메모리 세이브에 효과적

In [32]:
# yield는 제너레이터 처럼 하나씩 리턴값을 뽑음
def gen():
    for i in range(5):
        yield i *3
        
def gen2():
    for i in range(5):
        return i *3

for x in gen():
    print(x)

0
3
6
9
12


In [48]:
def average_hash(fname, size=16):
    img = Image.open(fname)
    img = img.convert('L')
    img = img.resize((size, size), Image.ANTIALIAS)
    pixel_data = img.getdata()
    pixels = np.array(pixel_data)
    pixels = pixels.reshape((size,size))
    avg = pixels.mean()
    diff = 1*(pixels > avg)
    return diff

ahash = average_hash('./img/ele.JPG')
print(ahash)

[[0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1]
 [1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0]
 [1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1]
 [1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## 유사 이미지 검색

In [74]:
search_dir = "./img/101_ObjectCategories/elephant"
cache_dir = "./img/cache_avghash"

if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)
    
def average_hash(fname, size= 16):
    fname2 = fname[len(search_dir): ]
    cache_file = cache_dir + '/' + fname2.replace('.jpg', '') + '.csv'
    if not os.path.exists(cache_file):
        img = Image.open(fname)
        img = img.convert('L').resize((size,size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt="%f", delimiter=',')
    else:
        px = np.loadtxt(cache_file, delimiter=",")
    return px

def hamming_dist(a, b):
    aa = a.reshape(1, -1)
    ab = b.reshape(1, -1)
    dist = (aa != ab).sum()
    return dist

def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root, f)
            if re.search(r'`.(jpg|jpeg|png)$', fname):
                yield fname
                
def find_image(fname, rate):
    src = average_hash(fname)
    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst) / 256
        if diff_r < rate:
            yield (diff_r, fname)
            
srcfile = search_dir + '/image_0001.jpg'
html = ""
sim = list(find_image(srcfile, 0.32))
sim = sorted(sim, key=lambda x: x[0])
for r, f in sim:
    print(r, ">", f) 
    s = '<div>' \
        '<h3>[ 차이 :' + str(r) + '-' + os.path.basename(f) + ']</h3>' + '<p><a href="' + f + '"><img src="' + f + '" width=400>' + '</a></p>' + '</div>'
    html += s

# HTML 파일로 출력하고 저장한다.
html = """
<html>
<head><meta charset="utf8"></head>
<body>
<h3>원래 이미지</h3>
<p><img src='{0}' width=400></p>{1}
</body>
</html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
    f.write(html) 
print("ok")

ok


In [75]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
#이미지 세트에 있는 디렉토리 이름으로 분류 대상의 카테고리를 선택한다.
caltech_dir = "./img/101_ObjectCategories"
categories = ["chair","camera"] 
nb_classes = len(categories)
image_w = 64 
image_h = 64
#색상 데이터를 나타내기 위해 각 픽셀마다 RGB값을 나타내는 3개의 데이터가 필요하므로 1개의 이미지는 3x64x64로 나타낸다.
pixels = image_w * image_h * 3 
# 실제 이미지 데이터를 저장한다.
X = []
#이미지가 어떤 것을 나타내는지 설명하는 레이블 데이터를 저장한다.
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 --- (※4)
    label = [0 for i in range(nb_classes)] 
    label[idx] = 1
    # 이미지 데이터를 검색한다.
    image_dir = caltech_dir + "/" + cat 
    files = glob.glob(image_dir+"/*.jpg") 
    for i, f in enumerate(files):
        img = Image.open(f)
        #이미지 파일을 읽고 색상 모드를 RGB로 변환한다.
        img = img.convert("RGB")
        img = img.resize((image_w, image_h)) 
        data = np.asarray(img) 
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y) 
xy = (X_train, X_test, y_train, y_test) 
np.save("./img/5obj.npy", xy)
print("ok,", len(Y))

0 
 [[[ 41  26  33]
  [ 33  18  23]
  [ 27  15  19]
  ...
  [ 81  73  97]
  [ 77  68  95]
  [ 70  61  90]]

 [[ 42  27  34]
  [ 36  21  26]
  [ 33  21  25]
  ...
  [ 83  75  99]
  [ 73  64  91]
  [ 71  62  91]]

 [[ 42  27  34]
  [ 35  20  25]
  [ 34  22  26]
  ...
  [ 84  76 100]
  [ 82  73 100]
  [ 74  65  94]]

 ...

 [[204 177 210]
  [215 194 225]
  [200 187 215]
  ...
  [207 177 211]
  [203 173 207]
  [200 170 204]]

 [[148 122 157]
  [136 115 148]
  [130 113 145]
  ...
  [200 170 204]
  [194 164 198]
  [192 162 196]]

 [[104  87 121]
  [163 141 177]
  [231 205 242]
  ...
  [194 164 198]
  [199 169 203]
  [193 163 197]]]
10 
 [[[227 217 205]
  [227 217 205]
  [227 217 205]
  ...
  [221 208 199]
  [221 208 199]
  [221 208 199]]

 [[227 217 205]
  [227 217 205]
  [229 219 207]
  ...
  [222 209 200]
  [221 208 199]
  [221 208 199]]

 [[229 219 207]
  [229 219 207]
  [230 220 208]
  ...
  [223 210 201]
  [223 210 201]
  [221 208 199]]

 ...

 [[201 186 183]
  [204 189 186]
  [202 187 

In [76]:
import logging
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import os
logging.disable(logging.WARNING)
# 카테고리를 지정한다.
categories = ["chair","camera"]
nb_classes = len(categories)
# 이미지 크기를 지정한다.
image_w = 64
image_h = 64
# 이미지를 변환한 데이터를 불러온다.
X_train, X_test, y_train, y_test = np.load("./img/5obj.npy", allow_pickle=True)
# 데이터를 정규화한다.
X_train = X_train.astype("float") / 256
X_test = X_test.astype("float") / 256
print('X_train shape:', X_train.shape)
#합성곱층, 활성화 함수, 맥스 풀링층, 전결합층으로 구성된 CNN 모델을 생성한다.
model = Sequential( )
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:])) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten( ))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, y_train, batch_size=32, epochs=50)
pre = model.predict(X_test)
score = model.evaluate(X_test, y_test)
print('손실=', score[0])
print('정확도=', score[1])

# 잘못 분류된 이미지를 저장한다.
for i, v in enumerate(pre): 
    pre_ans = v.argmax( ) 
    ans = y_test[i].argmax( ) 
    dat = X_test[i]
    if ans == pre_ans: 
        continue
    print("[NG]", categories[pre_ans], "!=", categories[ans])
    print(v)
    fname = "./img/error/" + str(i) + "-" + categories[pre_ans] + "-ne-" + categories[ans] + ".png" 
    print("이미지 저장", fname)
    dat *= 256
    # numpy 배열을 이미지로 변환한다.
    img = Image.fromarray(np.uint8(dat))
    img.save(fname)

X_train shape: (78, 64, 64, 3)
Epoch 1/50
78/78 [==============================] - 2s 20ms/step - loss: 2.7212 - acc: 0.4359
Epoch 2/50
78/78 [==============================] - 1s 9ms/step - loss: 0.7113 - acc: 0.5256
Epoch 3/50
78/78 [==============================] - 1s 8ms/step - loss: 0.9530 - acc: 0.5513
Epoch 4/50
78/78 [==============================] - 1s 8ms/step - loss: 0.6744 - acc: 0.6538
Epoch 5/50
78/78 [==============================] - 1s 8ms/step - loss: 0.5896 - acc: 0.6282
Epoch 6/50
78/78 [==============================] - 1s 8ms/step - loss: 0.5215 - acc: 0.8333
Epoch 7/50
78/78 [==============================] - 1s 8ms/step - loss: 0.5013 - acc: 0.8077
Epoch 8/50
78/78 [==============================] - 1s 8ms/step - loss: 0.4391 - acc: 0.8077
Epoch 9/50
78/78 [==============================] - 1s 8ms/step - loss: 0.3754 - acc: 0.8590
Epoch 10/50
78/78 [==============================] - 1s 8ms/step - loss: 0.3387 - acc: 0.8718
Epoch 11/50
78/78 [==================